In [1]:
from math import sin, radians, sqrt

import cadquery as cq
from cadquery import exporters
from jupyter_cadquery.replay import (replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import show, set_sidecar, set_defaults, open_viewer

import ipywidgets as widgets

Overwriting auto display for cadquery Workplane and Shape


In [2]:
open_viewer("adapter")
replay_box = widgets.Checkbox(False, description='Enable Replay')
out = widgets.Output()
display(replay_box)
display(out)

@out.capture()
def changed(e):
    if e["new"]:
        enable_replay(warning=False)
    else:
        disable_replay()

replay_box.observe(changed, names='value')

Checkbox(value=False, description='Enable Replay')

Output()

In [3]:
fan_hole = 120
fan_mount_width = 125
fan_screw_spacing = 112
screw_d = 4
screw_head_d = 8
insert_nut_d = 4.5

thickness = 2.5
adapter_body_height = 30

filter_length = 108
filter_width1 = 59
filter_width2 = 75
filter_holder_depth = 8
filter_holder_edge = 2.5

filter_mount_screw_spacing = 95
filter_mount_screw_d = 3

filter_pts = [(0, filter_width2/2),
              (filter_length/2, filter_width1/2),
              (filter_length/2, -filter_width1/2),
              (0, -filter_width2/2),
              (-filter_length/2, -filter_width1/2),
              (-filter_length/2, filter_width1/2),
              (0, filter_width2/2)]

screw_holes = [(fan_screw_spacing/2, fan_screw_spacing/2),
               (-fan_screw_spacing/2, fan_screw_spacing/2),
               (fan_screw_spacing/2, -fan_screw_spacing/2),
               (-fan_screw_spacing/2, -fan_screw_spacing/2)]

filter_mount_screw_holes = [(filter_mount_screw_spacing/2, filter_mount_screw_spacing/2),
                            (-filter_mount_screw_spacing/2, filter_mount_screw_spacing/2),
                            (filter_mount_screw_spacing/2, -filter_mount_screw_spacing/2),
                            (-filter_mount_screw_spacing/2, -filter_mount_screw_spacing/2)]

In [4]:
filter_2d = (
    cq.Workplane("XY")
    .moveTo(0, filter_width2/2)
    .lineTo(filter_length/2, filter_width1/2)
    .line(0, -filter_width1/2)
)
replay(filter_2d)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
  


In [5]:
filter_2d = cq.Sketch().polygon(filter_pts).vertices().fillet(5)
filter_shell_2d = filter_2d.copy().wires().offset(thickness).clean()
filter_inner_2d = (
    cq.Sketch()
    .polygon(filter_pts, tag="x")
    .wires().offset(-filter_holder_edge, mode="i")
    .select("x").delete()
    .vertices().fillet(5-filter_holder_edge)
)
mount_2d = cq.Sketch().rect(fan_mount_width, fan_mount_width)
fan_hole_2d = cq.Sketch().circle(fan_hole/2)

replay(filter_inner_2d)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
  


In [6]:
body_void = (
    cq.Workplane()
    .placeSketch(
        fan_hole_2d,
        filter_inner_2d.moved(cq.Location(cq.Vector(0, 0, adapter_body_height-filter_holder_depth-thickness)))
    )
    .loft()
    .faces(">Z").workplane()
    .placeSketch(filter_inner_2d)
    .extrude(thickness)
    .faces(">Z").workplane()
    .placeSketch(filter_2d)
    .extrude(filter_holder_depth)
)
replay(body_void)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
 


In [7]:
body = (
    cq.Workplane()
    .placeSketch(
        mount_2d,
        mount_2d.moved(cq.Location(cq.Vector(0, 0, thickness))),
        filter_shell_2d.moved(cq.Location(cq.Vector(0, 0, adapter_body_height)))
    )
    .loft(ruled=True)
    .cut(body_void)
    #############
    .faces("<Z").workplane()
    .pushPoints(screw_holes)
    .hole(screw_d)
    .faces("<Z").workplane(offset=-thickness)
    .pushPoints(screw_holes)
    .hole(screw_head_d)
)
replay(body)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
 


In [8]:
body = (
    cq.Workplane()
    .placeSketch(
        mount_2d,
    )
    .extrude(10)
    #.cut(body_void)
    #############
    .faces(">Z").workplane()
    .pushPoints(screw_holes)
    .cskHole(screw_d, screw_head_d, 82)
    ###############
    .faces(">Z").workplane()
    .pushPoints(filter_mount_screw_holes)
    .hole(insert_nut_d)
    ###############
    .faces(">Z").workplane()
    .hole(fan_hole)
)
replay(body)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
 


In [9]:
holder_base_2d = (
    cq.Sketch()
    .circle(fan_hole/2)
    .rect(filter_mount_screw_spacing, filter_mount_screw_spacing)
    .vertices()
    .circle(filter_mount_screw_d)
)
holder_base = (
    cq.Workplane()
)

replay(holder_base_2d)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
  


In [10]:
points = []
for i in range(15):
    points.extend([(0, 10*i), (0, -10*i)])

supports = (
    cq.Workplane()
    .pushPoints(points)
    .rect(filter_length, 1)
    .extrude(filter_holder_depth)
    .intersect(cq.Workplane().placeSketch(filter_2d).extrude(filter_holder_depth))
    .cut(cq.Workplane().rect(10, filter_width2).extrude(filter_holder_depth))
)

replay(supports)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
.

In [11]:
outlet_2d = cq.Sketch().circle(10).moved(cq.Location(cq.Vector(filter_length/2-10, 0)))

holder = (
    cq.Workplane()
    .placeSketch(filter_2d)
    .extrude(filter_holder_depth)
    .faces(">Z")
    .shell(-thickness)
    .faces("<Z")
    .rect(40, 40)
    .cutThruAll()
    .add(supports)
)

replay(holder)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
  


In [12]:
exporters.export(body, '/tmp/body.stl')
exporters.export(holder, '/tmp/holder.stl')